# Testing WOFpy SOAP and REST for WOF/WaterML 1.1
Testing performed on cloud-based WOFpy endpoints with ODM2 timeseries MySQL and PostgreSQL backends. These resources are managed by WOFpy developers for WOFpy testing and validation.

8/19/2017. Don Setiawan and Emilio Mayorga

In [1]:
from urlparse import urljoin
import io

import requests
import ulmo

In [2]:
print(ulmo.cuahsi.wof.__doc__)


    ulmo.cuahsi.wof
    ~~~~~~~~~~~~~~~

    `CUAHSI WaterOneFlow`_ web services

    .. _CUAHSI WaterOneFlow: http://his.cuahsi.org/wofws.html



In [3]:
print([obj for obj in dir(ulmo.cuahsi.wof) if not obj.startswith('__')])

['absolute_import', 'core', 'get_site_info', 'get_sites', 'get_values', 'get_variable_info']


## Endpoints and testing resources

In [4]:
def site_and_variable_codes(networkcode):
    if 'postgres' in networkcode:
        sitecode = '{netcode}:160065_4vars'.format(netcode=networkcode)
        variablecode = '{netcode}:EnviroDIY_Mayfly_Temp'.format(netcode=networkcode)
    elif 'mysql' in networkcode:
        sitecode = '{netcode}:USU-LBR-Mendon'.format(netcode=networkcode)
        variablecode = '{netcode}:USU36'.format(netcode=networkcode)
    
    return sitecode, variablecode

In [5]:
server_base_url = 'http://54.186.36.247:8080'
networkcodes = ['postgresqlodm2timeseries', 'mysqlodm2timeseries']

## SOAP 1.1 Testing

In [6]:
wofpy_wsdl_url = '/soap/cuahsi_1_1/.wsdl'
for networkcode in networkcodes:
    url = urljoin(server_base_url, networkcode+wofpy_wsdl_url)
    sitecode, variablecode = site_and_variable_codes(networkcode)

    print('--------------------------')
    print('Testing {}'.format(url))
    print('--------------------------')
        
    print('SITES-----------------')
    try:
        sites = ulmo.cuahsi.wof.get_sites(url)
        
        print(sites.keys())
    except:
        print('GetSites failed!')
    
    print('SITE INFO {}-------------'.format(sitecode))
    try:
        siteinfo = ulmo.cuahsi.wof.get_site_info(url, site_code=sitecode)
        
        print(siteinfo['site_property'])
    except:
        print('GetSiteInfo failed!')
    
    print('VARIABLES-------------')
    try:
        variables = ulmo.cuahsi.wof.get_variable_info(url)
        
        print(variables.keys())
    except:
        print('GetVariables failed!')

    print('VARIABLE INFO {}-------------'.format(variablecode))
    try:
        variable = ulmo.cuahsi.wof.get_variable_info(url, variable_code=variablecode)
        
        print(variable['name'])
    except:
        print('GetVariableInfo failed!')
    
        
    print('VALUES----------------')    
    try:
        values = ulmo.cuahsi.wof.get_values(url, site_code=sitecode, variable_code=variablecode)
        
        print(len(values['values']))
    except:
        print('GetValues failed!')

--------------------------
Testing http://54.186.36.247:8080/postgresqlodm2timeseries/soap/cuahsi_1_1/.wsdl
--------------------------
SITES-----------------
GetSites failed!
SITE INFO postgresqlodm2timeseries:160065_4vars-------------
GetSiteInfo failed!
VARIABLES-------------
GetVariables failed!
VARIABLE INFO postgresqlodm2timeseries:EnviroDIY_Mayfly_Temp-------------
GetVariableInfo failed!
VALUES----------------
GetValues failed!
--------------------------
Testing http://54.186.36.247:8080/mysqlodm2timeseries/soap/cuahsi_1_1/.wsdl
--------------------------
SITES-----------------
['mysqlodm2timeseries:USU-LBR-Mendon', 'mysqlodm2timeseries:USU-LBR-SFWeather']
SITE INFO mysqlodm2timeseries:USU-LBR-Mendon-------------
{'site_comments': 'Located below county road bridge at Mendon Road crossing'}
VARIABLES-------------
['mysqlodm2timeseries:USU57', 'mysqlodm2timeseries:USU56', 'mysqlodm2timeseries:USU55', 'mysqlodm2timeseries:USU54', 'mysqlodm2timeseries:USU53', 'mysqlodm2timeseries:US

## REST 1.1 Testing

In [7]:
wofpy_rest_url = '/rest/1_1/'
for networkcode in networkcodes:
    url = urljoin(server_base_url, networkcode+wofpy_rest_url)
    sitecode, variablecode = site_and_variable_codes(networkcode)

    print('--------------------------')
    print('Testing {}'.format(url))
    print('--------------------------')
    
    print('SITES-----------------')
    try:
        req = requests.get(urljoin(url, 'GetSites'))
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        sites = ulmo.waterml.v1_1.parse_site_infos(response_buffer)
        
        print(sites.keys())
    except:
        print('GetSites failed!')
    
    print('SITE INFO {}-------------'.format(sitecode))
    try:
        req = requests.get(urljoin(url, 'GetSiteInfo'), params={'site':sitecode})
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        siteinfo = ulmo.waterml.v1_1.parse_sites(response_buffer)
        
        print(siteinfo[sitecode.split(':')[1]]['site_property'])
    except:
        print('GetSiteInfo failed!')
    
    print('VARIABLES-------------')
    try:
        req = requests.get(urljoin(url, 'GetVariables'))
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        variables = ulmo.waterml.v1_1.parse_variables(response_buffer)
        
        print(variables.keys())
    except:
        print('GetVariables failed!')

    print('VARIABLE INFO {}-------------'.format(variablecode))
    try:
        req = requests.get(urljoin(url, 'GetVariableInfo'), params={'variable':variablecode})
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        variable = ulmo.waterml.v1_1.parse_variables(response_buffer)
        
        print(variable[variablecode.split(':')[1]]['name'])
    except:
        print('GetVariableInfo failed!')
    
        
    print('VALUES----------------')    
    try:
        req = requests.get(urljoin(url, 'GetValues'), params={'location':sitecode, 
                                                              'variable':variablecode})
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        values = ulmo.waterml.v1_1.parse_site_values(response_buffer)
        
        print(len(values[variablecode.split(':')[1]]['values']))
    except:
        print('GetValues failed!')

--------------------------
Testing http://54.186.36.247:8080/postgresqlodm2timeseries/rest/1_1/
--------------------------
SITES-----------------
[]
SITE INFO postgresqlodm2timeseries:160065_4vars-------------
GetSiteInfo failed!
VARIABLES-------------
[]
VARIABLE INFO postgresqlodm2timeseries:EnviroDIY_Mayfly_Temp-------------
GetVariableInfo failed!
VALUES----------------
GetValues failed!
--------------------------
Testing http://54.186.36.247:8080/mysqlodm2timeseries/rest/1_1/
--------------------------
SITES-----------------
['USU-LBR-Mendon', 'USU-LBR-SFWeather']
SITE INFO mysqlodm2timeseries:USU-LBR-Mendon-------------
{'site_comments': 'Located below county road bridge at Mendon Road crossing'}
VARIABLES-------------
['USU36', 'USU35', 'USU44', 'USU47', 'USU48', 'USU49', 'USU30', 'USU26', 'USU27', 'USU24', 'USU25', 'USU22', 'USU23', 'USU20', 'USU21', 'USU62', 'USU31', 'USU60', 'USU61', 'USU28', 'USU29', 'USU7', 'USU6', 'USU5', 'USU4', 'USU3', 'USU33', 'USU32', 'USU9', 'USU8', '